##Información sobre el conjunto de datos:

Los datos se extrajeron de imágenes tomadas de ejemplares auténticos y falsificados de billetes. Las imágenes finales tienen 400x 400 píxeles. Se obtuvieron imágenes en escala de grises con una resolución de unos 660 ppp. Se utilizó la herramienta de Transformación Wavelet para extraer las características de las imágenes.

Hay 4 variables de predicción (varianza de la imagen, asimetría, curtosis, entropía). La variable a predecir se codifica como 0 (auténtica) o 1 (falsa). Véase https://archive.ics.uci.edu/ml/datasets/banknote+authentication.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#import h5py

import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
from tensorflow.keras.activations import sigmoid

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


In [2]:
from google.colab import files
uploaded = files.upload()

Saving data_banknote_authentication.txt to data_banknote_authentication.txt


In [3]:
# cargar los datos
data_df = pd.read_csv("data_banknote_authentication.txt",
                           names=['Varianza',
                                  'Asimetría',
                                  'Curtosis',
                                  'Entropía', 
                                  'Clase'])
data_df

,Varianza,Asimetría,Curtosis,Entropía,Clase
0,3.62160,8.66610,-2.8073,-0.44699,0
1,4.54590,8.16740,-2.4586,-1.46210,0
2,3.86600,-2.63830,1.9242,0.10645,0
3,3.45660,9.52280,-4.0112,-3.59440,0
4,0.32924,-4.45520,4.5718,-0.98880,0
...,...,...,...,...,...
1367,0.40614,1.34920,-1.4501,-0.55949,1
1368,-1.38870,-4.87730,6.4774,0.34179,1
1369,-3.75030,-13.45860,17.5932,-2.77710,1
1370,-3.56370,-8.38270,12.3930,-1.28230,1


In [4]:
# Separar en características y etiquetas
data_x = data_df[['Varianza','Asimetría','Curtosis','Entropía']].to_numpy()
data_y = data_df[['Clase']].to_numpy()


In [5]:
# Segmentar en ENTRENAMIENTO y prueba
TRAIN_x_orig, test_x_orig, TRAIN_y, test_y = train_test_split(data_x, data_y, test_size=0.20, random_state=42)


In [6]:
# Dimensiones de los conjuntos de ENTRENAMIENTO y prueba
print ("TRAIN_x_orig shape: " + str(TRAIN_x_orig.shape))
print ("TRAIN_y shape: " + str(TRAIN_y.shape))
print ("test_x_orig shape: " + str(test_x_orig.shape))
print ("test_y shape: " + str(test_y.shape))

TRAIN_x_orig shape: (1097, 4)
TRAIN_y shape: (1097, 1)
test_x_orig shape: (275, 4)
test_y shape: (275, 1)


In [7]:
# Escalamiento de los datos
scaler = MinMaxScaler()
scaler.fit(TRAIN_x_orig)

TRAIN_x = scaler.transform(TRAIN_x_orig)
test_x = scaler.transform(test_x_orig)


In [8]:
# Segmentación en entrenamiento y validación
train_x, valid_x, train_y, valid_y = train_test_split(TRAIN_x, TRAIN_y, test_size=0.2, random_state=42)


In [9]:
# Resumen de dimensiones
print ("TRAIN_x shape: " + str(TRAIN_x.shape))
print ("TRAIN_y shape: " + str(TRAIN_y.shape))
print ("train_x shape: " + str(train_x.shape))
print ("train_y shape: " + str(train_y.shape))
print ("valid_x shape: " + str(valid_x.shape))
print ("valid_y shape: " + str(valid_y.shape))
print ("test_x shape:  " + str(test_x.shape))
print ("test_y shape:  " + str(test_y.shape))

TRAIN_x shape: (1097, 4)
TRAIN_y shape: (1097, 1)
train_x shape: (877, 4)
train_y shape: (877, 1)
valid_x shape: (220, 4)
valid_y shape: (220, 1)
test_x shape:  (275, 4)
test_y shape:  (275, 1)


In [24]:
# Creación del modelo
model = Sequential([
                    Dense(units=10, input_shape= [train_x.shape[-1]], activation=sigmoid)
                    ])

In [25]:
# Arquitectura del modelo
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 10)                50        
Total params: 50
Trainable params: 50
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Definición del optimizador, función de pérdidas y métricas
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss= tf.keras.losses.BinaryCrossentropy(),metrics=tf.keras.metrics.BinaryAccuracy() )


In [27]:
model.fit(train_x, train_y, epochs=200, batch_size=32)

Epoch 1/200


ValueError: ignored

In [14]:
# Realizar predicciones
train_pred_prob = model.predict(train_x)
valid_pred_prob = model.predict(valid_x)

In [15]:
# Predicción de clases
train_pred_class = train_pred_prob>0.5
valid_pred_class = valid_pred_prob>0.5

In [16]:
# Metricas
Metricas = model.evaluate(train_x, train_y , verbose=0)
Metricas

[0.4592219293117523, 0.8346636295318604]

In [17]:
# Accuracy
acc_train = accuracy_score(train_y, train_pred_class)
acc_valid = accuracy_score(valid_y, valid_pred_class)

print('Train Acc: ', acc_train)
print('Validation Acc: ', acc_valid)

Train Acc:  0.8346636259977195
Validation Acc:  0.8227272727272728


In [18]:
# Creación del modelo
modelo_final = Sequential([
                    Dense(units=1, input_shape= [TRAIN_x.shape[-1]], activation=sigmoid)
                    ])

# Arquitectura del modelo
modelo_final.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 5         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Definición del optimizador, función de pérdidas y métricas
modelo_final.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss= tf.keras.losses.BinaryCrossentropy(),metrics=tf.keras.metrics.BinaryAccuracy() )


In [20]:
modelo_final.fit(TRAIN_x, TRAIN_y, epochs=200, batch_size=32)

Epoch 1/200
35/35 [==============================] - 0s 2ms/step - loss: 0.8981 - binary_accuracy: 0.4370
Epoch 2/200
35/35 [==============================] - 0s 2ms/step - loss: 0.8468 - binary_accuracy: 0.4691
Epoch 3/200
35/35 [==============================] - 0s 2ms/step - loss: 0.8301 - binary_accuracy: 0.4572
Epoch 4/200
35/35 [==============================] - 0s 2ms/step - loss: 0.8207 - binary_accuracy: 0.4336
Epoch 5/200
35/35 [==============================] - 0s 2ms/step - loss: 0.8019 - binary_accuracy: 0.4109
Epoch 6/200
35/35 [==============================] - 0s 2ms/step - loss: 0.7860 - binary_accuracy: 0.3851
Epoch 7/200
35/35 [==============================] - 0s 2ms/step - loss: 0.7816 - binary_accuracy: 0.3229
Epoch 8/200
35/35 [==============================] - 0s 2ms/step - loss: 0.7660 - binary_accuracy: 0.2958
Epoch 9/200
35/35 [==============================] - 0s 2ms/step - loss: 0.7608 - binary_accuracy: 0.2509
Epoch 10/200
35/35 [==========================

In [21]:
# Realizar predicciones
TRAIN_pred_prob = modelo_final.predict(TRAIN_x)
test_pred_prob = modelo_final.predict(test_x)

# Predicción de clases
TRAIN_pred_class = TRAIN_pred_prob>0.5
test_pred_class = test_pred_prob>0.5

In [22]:
# Accuracy
acc_TRAIN = accuracy_score(TRAIN_y, TRAIN_pred_class)
acc_test = accuracy_score(test_y, test_pred_class)

print('TRAIN Acc: ', acc_TRAIN)
print('test Acc: ', acc_test)

TRAIN Acc:  0.837739288969918
test Acc:  0.8072727272727273
